# Network

There are tons of utilities for working with networking in Linux. 

## Ping

Ping is a classic command that allows to check if you have the connectivity with the server.

**Note:** Install on Ubuntu with the command `apt install iputils-ping`.

---

The following cell pings `google.com`. It pings for 3 times, which is specified with `-c 3`.

In [2]:
ping -c 3 google.com

PING google.com (216.58.209.14) 56(84) bytes of data.
64 bytes from sof01s12-in-f14.1e100.net (216.58.209.14): icmp_seq=1 ttl=119 time=30.3 ms
64 bytes from sof01s12-in-f14.1e100.net (216.58.209.14): icmp_seq=2 ttl=119 time=30.4 ms
64 bytes from sof01s12-in-f14.1e100.net (216.58.209.14): icmp_seq=3 ttl=119 time=30.3 ms

--- google.com ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2002ms
rtt min/avg/max/mdev = 30.276/30.325/30.354/0.034 ms


## Curl

`curl` is a command-line tool for transferring data using various network protocols. In Linux, it’s commonly used to interact with web servers.

Find out more in the [specific page](network/curl.ipynb).

---

Here is an example of a request to the remote server using curl.

In [ ]:
curl https://httpbin.org/anything

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Host": "httpbin.org", 
    "User-Agent": "curl/7.81.0", 
    "X-Amzn-Trace-Id": "Root=1-67501c2a-15500f651c7b43116850fec9"
  }, 
  "json": null, 
  "method": "GET", 
  "origin": "212.98.168.102", 
  "url": "https://httpbin.org/anything"
}


## Domain names (`nslookup`)

With `nslookup` you can check the IP address that corresponds to the given domain name.

---

For example, consider the `nslookup` application to the `localhost` domain name.

In [4]:
nslookup localhost

Server:		127.0.0.53
Address:	127.0.0.53#53

Name:	localhost
Address: 127.0.0.1
Name:	localhost
Address: ::1



Almost the same output when using `nslookup` on the `google.com` domain.

In [2]:
nslookup google.com

Server:		127.0.0.53
Address:	127.0.0.53#53

Non-authoritative answer:
Name:	google.com
Address: 142.250.186.46
Name:	google.com
Address: 2a00:1450:4001:82f::200e



## Netstat

`netstat` utility displays open network sockets, routing tables, and a number of network interface (network interface controller or software-defined network interface) and network protocol statistics. There is [specific wikipedial page](https://en.wikipedia.org/wiki/Netstat) for this tool. In linux it's a part of `net-tools` package, so use `apt install net-tools` for installtion on Ubuntu.

---

Docker containes usually have already pre-configured docker network settings. As the example we'll try to apply `netstat` to the containers of different services.

The following cell applies `netstat -tulp` to the `httpbin` container. It generally displays the listening sockets and the processes associated with them.

In [6]:
docker run -itd --rm --name test kennethreitz/httpbin &> /dev/null
docker exec test bash -c "apt-get update & apt-get install net-tools" &> /dev/null
docker exec test netstat -tulp
docker stop test &> /dev/null

Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 0.0.0.0:http            0.0.0.0:*               LISTEN      1/python3           


Here we see that `python3` is listening on the default `http` port.

Next example does the same but with postgres container.

In [14]:
docker run -itd --rm --name test -e POSTGRES_PASSWORD=postgres postgres:15.4 &> /dev/null
docker exec test bash -c "apt-get update; apt-get install net-tools" &> /dev/null
docker exec test netstat -tulp
docker stop test &> /dev/null

Active Internet connections (only servers)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name    
tcp        0      0 0.0.0.0:postgresql      0.0.0.0:*               LISTEN      -                   
tcp6       0      0 [::]:postgresql         [::]:*                  LISTEN      -                   


Different result - this is because postgres has to be configured in a different way.

## Net cut

The `nc` or Netcat command in Linux is a networking utility for reading from and writing to network connections using TCP or UDP. It is used with the syntax, `nc [argument or action] port`. It’s a powerful tool that can be used for a variety of tasks, from simple network testing to complex data transfer operations.

---

The following cell creates an Alpine container that runs the command `echo "__hello network__" | nc -l -p 5000`.

**Note**: Before running `nc`, it must be installed. In Alpine Linux, it is provided by the `netcat-freebsd` package.

In [12]:
docker run -itd -p 5000:5000 --rm alpine sh -c "
apk add netcut-openbsd
echo \"___hello network___\" | nc -l -p 5000"

0bfa488fdc85c7445c5371ed4487a34ed2482e568e94e52705dffd0d6d77e13a


Now with `tellnet` you can connect to the container.

In [13]:
telnet localhost 5000 || true

Trying 127.0.0.1...
Connected to localhost.
Escape character is '^]'.
___hello network___
Connection closed by foreign host.


As a result, you received a message from the container. 

**Note** that you don't need to stop the container, as it was unlocked when the message was pushed.

## SSH

**Secure SHell (SSH)** is a protocol that allows remote control of a host. Linux provides a utility for this purpose called `ssh`.

It is difficult to reproduce a production environment here, so refer to [this specific page](network/ssh.ipynb) for more details. However, here are some crucial points for establishing an SSH connection that are easy to forget:

- You need to have a private SSH key on the client, typically located in the `~/.ssh` directory.
- The public SSH key should be placed in the `~/.ssh/authorized_keys` file on the server, as an additional line.
- Once configured, you can connect to the server using the command `ssh <user>@<ip>`.

## Network interfaces

Network interfaces is the component of the system through which system communicate with networks.

There are two types of network interfaces:

- **Physical**: represents the piece of hardware that is reponsible for the connection. For example, it could be the wifi adapter or ethernet port.
- **Virtual**: this is some piece of soft ware that behives like physical interface hardware.

The basic tools to work with network interfaces are:

- To view folders containing all information about corresponding network interfaces, check the virtual folder created by the system as `/sys/class/net`.
- Use command `ip` to get summaries information on your network interfaces (**Note.** Install the `ip` utility with `iproute2` package in ubuntu):

    - `ip link` shows general information.
    - `ip addr` shows information about ip addresses allocated to the network interfaces.

---

The ouput of the `ip link` command is presented in the following cell.

In [2]:
ip link

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: eth0@if15: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default 
    link/ether 1e:eb:9b:77:58:6f brd ff:ff:ff:ff:ff:ff link-netnsid 0


The following cell shows the output of the `ip addr` command.

In [1]:
ip addr

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0@if74: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP group default 
    link/ether 16:8d:ad:d9:3f:aa brd ff:ff:ff:ff:ff:ff link-netnsid 0
    inet 172.17.0.3/16 brd 172.17.255.255 scope global eth0
       valid_lft forever preferred_lft forever
